In [ ]:
import os
import lmdb
import pickle
import time
import pandas as pd
import numpy as np
from rdkit import Chem
from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  
import warnings
warnings.filterwarnings(action='ignore')


In [ ]:
date_str = time.strftime("%m%d-%y", time.localtime())

data_path='./'  # replace to your data path
save_dir=f"./save_4CR_{date_str}"  # replace to your save path
MASTER_PORT=10086
n_gpu=1
dict_name='dict.txt'
# weight_path='./weights/mol_pre_no_h_220816.pt'  # replace to your ckpt path, 0.81 when using unimol layers
# weight_path='/fs01/datasets/cellxgene/3d_molecule_save/pretrain-20240502-2040/checkpoint_best.pt'  # on 12 hour training, 0.75 when using unimol layers; 15h 0.77 on a100; 0.72 using pytorch layernorm; 18h 0.80 on a100
weight_path="/home/pangkuan/dev/SDL-LNP/model/evaluation/notebooks/model_weight/mol_pre_no_h_220816.pt"  # on 15 hour training, 0.82 when using unimol layers; 18h 0.84 on a100; 21h 0.82 on a100; 0.84 using pytorch softmax
# weight_path = "/home/pangkuan/dev/SDL-LNP/model/evaluation/notebooks/model_weight/customized_model/checkpoint_best.pt"
# weight_path='/fs01/datasets/cellxgene/3d_molecule_save/pretrain-20240503-2035/checkpoint_best.pt'
# task_name='4CR'  # data folder name
task_name="4CR_customized"
task_num=1
loss_func='finetune_mse'
lr=1e-4
batch_size=64
epoch=60
dropout=0.0
warmup=0.06 # 0.06
local_batch_size=64
only_polar=0 # -1 all h; 0 no h
conf_size=11
seed=0
metric="valid_agg_rmse"
update_freq=batch_size / local_batch_size
results_path=f"./infer_4CR_{date_str}"  # replace to your results path
weight_path=f"./save_4CR_{date_str}/checkpoint_best.pt"  # replace to your ckpt path
# batch_size=64
# task_name='4CR' # data folder name 
# task_num=1
# loss_func='finetune_mse'
# dict_name='dict.txt'
# conf_size=11
# only_polar=0

!cp ../example_data/molecule/$dict_name $data_path
!CUDA_VISIBLE_DEVICES="0" python ../unimol/infer.py --user-dir ../unimol $data_path --task-name $task_name --valid-subset test \
       --results-path $results_path \
       --num-workers 6 --ddp-backend=c10d --batch-size $batch_size \
       --task mol_finetune --loss $loss_func --arch unimol_base \
       --classification-head-name $task_name --num-classes $task_num \
       --dict-name $dict_name --conf-size $conf_size \
       --only-polar $only_polar  \
       --path $weight_path  \
       --fp16 --fp16-init-scale 4 --fp16-scale-window 256 \
       --log-interval 50 --log-format simple 